### 🗺 기능 시나리오
| 사용자 요청                 | Agent 동작                              |
| ---------------------- | ------------------------------------- |
| “서울에서 1박 2일 힐링 여행 알려줘” | ✅ 위치 해석 → 날씨 확인 → 관광지 추천 (일정별 구성)     |
| “이 장소는 별로야, 바꿔줘”       | ✅ 해당 지점 이후 일정만 재생성                    |
| “여기 자세히 알려줘” *(기본 포함)* | ✅ 장소별로 설명 + 즐길거리 + 리뷰 요약 포함           |
| “날씨 안 좋으면 실내만 추천해줘”    | ✅ 날씨 확인 → 실내 필터링 자동 적용                |
| “지도에 표시해줘”             | ✅ 장소별 `위도(lat)`, `경도(lng)` 포함 JSON 반환 |
| “정보가 부족해” *(기본 포함)*    | ✅ API 또는 검색 결과로 보강된 설명 자동 제공          |


### 🧾응답 포맷 (기본 제공 정보 포함)
```json
{
  "summary": "서울 1박 2일 힐링 여행 코스입니다. 비 예보로 실내 위주로 구성했습니다.",
  "weather": {
    "day1": "흐림",
    "day2": "비"
  },
  "route": [
    {
      "day": 1,
      "places": [
        {
          "name": "국립현대미술관 서울관",
          "lat": 37.5796,
          "lng": 126.9818,
          "description": "한국 현대미술을 감상할 수 있는 실내 전시관",
          "highlights": ["전시관 1~4관", "미디어아트존", "뮤지엄숍"],
          "reviews_summary": "방문객들은 전시 품질이 우수하며 조용한 분위기를 선호함",
          "data_source": "한국관광공사 API + 웹 검색 보완"
        },
        ...
      ]
    }
  ]
}

```

### ✅ 핵심 처리 흐름
1. 사용자 입력 분석
    - 위치/일정/테마 파악
2. 날씨 확인
    - 날짜별로 실내 필터링 적용 여부 결정
3. 장소 추천
    - TourAPI 기반 추천 + 부족 정보는 검색 자동 보완
4. 장소 정보 구성
    - 설명 / 즐길거리 / 리뷰 요약 기본 제공
5. 일정 구성
    - 날짜별 장소/시간 구성
6. 카카오 지도용 좌표 포함 반환

### 🧠 Agent 핵심 도구 목록 (LangChain 기준)
| Tool 이름             | 설명                                    |
| ------------------- | ------------------------------------- |
| `LocationResolver`  | 도시명, 현재 위치 → 지역 코드 + 위경도 변환           |
| `WeatherChecker`    | 일정 기간 날씨 예측                           |
| `TourAPIClient`     | 관광지 후보 추천 (지역 기반 + 테마 기반)             |
| `IndoorFilter`      | 날씨 나쁠 경우 실내 장소만 추출                    |
| `PlaceExplainer`    | **기본 제공**: 장소 설명, 즐길거리, 사용자 리뷰 요약     |
| `WebSearchFallback` | **기본 제공**: 설명 부족 시 검색으로 정보 보완         |
| `RoutePlanner`      | 여행 일정 기반 코스 자동 생성                     |
| `MapPointFormatter` | 카카오지도 마커 포맷 반환 (`lat`, `lng`, `name`) |


### 구현
- 지역명 → 지역 코드/좌표 변환기 만들기
- 날씨 조회기 (OpenWeatherMap 또는 기상청 API)
- TourAPI → 장소 후보 가져오기
- 실내 필터링 조건 구성 (cat3 or 실내 여부 파악 로직)
- 일정 기반 경로 자동 구성 로직
- 장소 설명 + 리뷰 요약용 Scraper or API 연동
- 지도 포맷 JSON 반환기
- LangChain Agent로 통합 (Tools 묶기)



국문관광정보API: https://www.data.go.kr/data/15101578/openapi.do 